In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from datasets import load_dataset
from transformers import (DataCollatorWithPadding,create_optimizer,DebertaTokenizerFast, DebertaV2ForQuestionAnswering)

2024-08-01 19:04:02.521135: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 19:04:02.521206: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 19:04:02.531105: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-01 19:04:02.600793: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-01 19:04:03.970284: W tensorflow/compiler/tf2

In [2]:
BATCH_SIZE=32
MAX_LENGTH=512

## Data Preperation

In [3]:
dataset=load_dataset("squad")


In [4]:
model_id="microsoft/deberta-base"
tokenizer=DebertaTokenizerFast.from_pretrained(model_id)

In [5]:
tokenized_examples = tokenizer(
    dataset['train'][0]['question'],
    dataset['train'][0]["context"],
    truncation="only_second",
    max_length=MAX_LENGTH,
    stride=64,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
)

In [6]:
sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
offset_mapping = tokenized_examples.pop("offset_mapping")

tokenized_examples["start_positions"] = []#[152,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
tokenized_examples["end_positions"] = []#[172,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [7]:
for i, offsets in enumerate(offset_mapping):
  if len(dataset['train'][0]['answers']['answer_start'])==0:
    tokenized_examples["start_positions"].append(0)
    tokenized_examples["end_positions"].append(0)
  else:
      
    start_char=dataset['train'][0]['answers']['answer_start'][0]
    end_char=start_char+len(dataset['train'][0]['answers']['text'][0])
    found=0
    start_token_position=0
    end_token_position=0
    
    for j,offset in enumerate(offsets):
      if offset[0]<=start_char and offset[1]>=start_char and found==0:
        start_token_position=j
        end_token_position=MAX_LENGTH
        
        found=1
      if offset[1]>=end_char and found==1:
        end_token_position=j
        break
    tokenized_examples["start_positions"].append(start_token_position)
    tokenized_examples["end_positions"].append(end_token_position)

In [8]:
print(tokenized_examples['start_positions'])
print(tokenized_examples['end_positions'])

[134]
[141]


In [9]:
def preprocess_function(dataset):
  
  questions = [q.lstrip() for q in dataset["question"]]
  paragraphs = [p.lstrip() for p in dataset["context"]]
  
  tokenized_examples = tokenizer(
    questions,
    paragraphs,
    truncation="only_second",
    max_length=MAX_LENGTH,
    stride=64,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
  )
  sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
  offset_mapping = tokenized_examples.pop("offset_mapping")

  tokenized_examples["start_positions"] = []#[152,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  tokenized_examples["end_positions"] = []#[172,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

  for i, offsets in enumerate(offset_mapping):
    sample_index = sample_mapping[i]

    start_char=dataset["answers"][sample_index]['answer_start'][0]
    end_char=start_char+len(dataset["answers"][sample_index]['text'][0])
    found=0
    start_token_position=0
    end_token_position=0
    
    for j,offset in enumerate(offsets):
      if offset[0]<=start_char and offset[1]>=start_char and found==0:
        start_token_position=j
        end_token_position=MAX_LENGTH
        found=1
      if offset[1]>=end_char and found==1:
        end_token_position=j
        break
    tokenized_examples["start_positions"].append(start_token_position)
    tokenized_examples["end_positions"].append(end_token_position)

  return tokenized_examples

In [10]:
tokenized_dataset=dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

In [11]:
tf_dataset = tokenized_dataset["train"].to_tf_dataset(
    shuffle=True, 
    batch_size=BATCH_SIZE,
)

2024-08-01 19:04:19.188800: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-01 19:04:19.189837: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-01 19:04:19.189911: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-01 19:04:19.192260: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-01 19:04:19.192352: I external/local_xla/xla/stream_executor

In [12]:
train_dataset=tf_dataset.take(int(0.9*len(tf_dataset)))

In [13]:
val_dataset=tf_dataset.skip(int(0.9*len(tf_dataset)))

## Modeling

In [14]:
from transformers import TFAutoModel
model = TFAutoModel.from_pretrained("microsoft/deberta-v2-xlarge")
model.summary()

2024-08-01 19:04:30.238140: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 750.59MiB (rounded to 787046400)requested by op StatelessTruncatedNormalV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-08-01 19:04:30.238219: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-08-01 19:04:30.238244: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 13, Chunks in use: 13. 3.2KiB allocated for chunks. 3.2KiB in use in bin. 109B client-requested in use in bin.
2024-08-01 19:04:30.238254: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-08-01 19:04:30.238

ResourceExhaustedError: {{function_node __wrapped__StatelessTruncatedNormalV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[128100,1536] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessTruncatedNormalV2] name: 

In [ ]:
optimizer=Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer)

In [ ]:
history=model.fit(train_dataset,validation_data=val_dataset,epochs=1)

## Evaluation

In [ ]:
question="How does the virus spread?"
text="We know that the disease is caused by the SARS-CoV-2 virus, which spreads between people in several different ways.Current evidence suggests that the virus spreads mainly between people who are in close contact with each other, for example at a conversational distance.The virus can spread from an infected person’s mouth or nose in small liquid particles when they cough, sneeze, speak, sing or breathe. Another person can then contract the virus when infectious particles that pass through the air are inhaled at short range (this is often called short-range aerosol or short-range airborne transmission) or if infectious particles come into direct contact with the eyes, nose, or mouth (droplet transmission). The virus can also spread in poorly ventilated and/or crowded indoor settings, where people tend to spend longer periods of time. This is because aerosols can remain suspended in the air or travel farther than conversational distance (this is often called long-range aerosol or long-range airborne transmission). People may also become infected when touching their eyes, nose or mouth after touching surfaces or objects that have been contaminated by the virus. Further research is ongoing to better understand the spread of the virus and which settings are most risky and why. Research is also under way to study virus variants that are emerging and why some are more transmissible. For updated information on SARS-CoV-2 variants, please read the weekly epidemiologic updates."
inputs = tokenizer(question, text, return_tensors="tf")
outputs = model(**inputs)

answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)